# Tools for auditing the Qdrant DB, finding and deleting documents


#### NOTE: please see end of notebook for data structure examples that is assumed in the search code

[documentation is here. Very hard to find online searching:
](https://qdrant.github.io/qdrant/redoc/index.html#tag/points/operation/scroll_points)


## 0. Imports and Environmental variables


In [1]:
# Ccnfirm you're in the right interpreter
#
import sys
print(sys.executable)

/Users/drew_wilkins/Drews_Files/Drew/Python/ASK/.venv-v1/bin/python


In [2]:
import pandas as pd
from datetime import datetime
import streamlit as st


# CONFIG: qdrant
api_key = st.secrets["QDRANT_API_KEY"]
url = st.secrets["QDRANT_URL"]  # for cloud
qdrant_collection_name = "ASK_vectorstore"

# for local instand. ON mac local instance is /private/tmp/local_qdrant
qdrant_path = "/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/data/qdrant"

## 1. Qdrant client


In [3]:
from qdrant_client import QdrantClient

# Load an instance of the client. WITHOUT LANGCHAIN
# 22.5 sec for cloud
# Running this places a lock file in the qdrant directory

client = QdrantClient(
    url=url,
    # prefer_grpc=True,
    api_key=api_key,
    # path=qdrant_path
)

content = (client.get_collections())
print(content)

collections=[CollectionDescription(name='ask_pdf_pages'), CollectionDescription(name='ASK_vectorstore'), CollectionDescription(name='ask_pdf_docs')]


2. ## Create a range index for `metadata.page`, so you can use this indexed field to order your results by page number.

In [ ]:
from qdrant_client.http import models

client.create_payload_index(
    collection_name=qdrant_collection_name,
    field_name="metadata.page",
    # Assuming page numbers are stored as integers
    field_schema=models.PayloadSchemaType.INTEGER,
)

UpdateResult(operation_id=237, status=<UpdateStatus.COMPLETED: 'completed'>)

### Optional: get all points


In [ ]:
# Gets all points (i.e., records) and returns them as a 2-tuple
# Tuple[List[types.Record], Optional[types.PointId]]
# 7.1 sec for cloud

all_records = client.scroll(
    collection_name=qdrant_collection_name,
    limit=100000,
    with_payload=True,  # change to True to see the payload
    with_vectors=False  # change to True to see the vectors
)

print(f"""all_records is a {type(all_records)} containing {len(all_records)}elements. 
    The first element is a {type(all_records[0])} of length {len(all_records[0])}).
    The second element is a {type(all_records[1])} of length unknown
    """)

all_records[0]  # see the first point#

## 2. Find points in the Qdrant DB


##### Find points based on a text-based metadata field (e.g., source, file_name, page_content)


In [ ]:
# uses MatchText to find points that contain text in source field.
# text is case sensitive
# the order_by part requires qdrant-clietn 1.8 or later

from qdrant_client.http import models

client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=1000,     # defaults to 10
    order_by=models.OrderBy(
        key="metadata.page",  # Field for ordering
        direction="asc"       # Change to "desc" for descending order
    ),
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.source",
                # key="metadata.page_content",
                # key="metadata.filename",
                match=models.MatchText(
                    text="AUXCA_SOP"),
            ),
        ]
    ),
)

([Record(id='a641ff3c-68ed-4cea-98fa-a927c6e5606b', payload={'metadata': {'page': 0, 'source': 'References/SOPs/AUXCA_SOP_FINAL_18JUN21_esign.pdf'}, 'page_content': 'United States Coast Guard Auxiliary  \n \n \n \n            \nAuxiliary Culinary Assistan ce (AUXCA)  \nProgram  \n \nStandard Operating Procedures'}, vector=None, shard_key=None, order_value=0),
  Record(id='b4658f4b-f88c-40d3-abf4-c6f3c0ceb315', payload={'metadata': {'page': 1, 'source': 'References/SOPs/AUXCA_SOP_FINAL_18JUN21_esign.pdf'}, 'page_content': ' \n \nComman dant \nU. S. Coast Guard  \n \n 2703 Martin Luther King Jr. Ave, SE  \nSTOP 7 501 \nWashington, DC 20593 -7501 \nStaff Symbol: CG-BSX-1 \nPhone: (202) 372 -1261 \n \n \n \n1679 0 / AUX -SOP-005(A)  \n18 Jun 21 \n \n  \nMEMORANDUM         Reply  to     S. Minutolo \n            Attn.  of:      (202) 372- 1267 \n \nFrom:  S.L. Johnson   CAPT /s/ \nCOMDT (CG -BSX)     \nTo: \n Distribution  \nSubj:  AUXILIARY CULINARY ASSISTAN CE (AUXCA)  PROGRAM STANDARD \n

In [ ]:
# uses MatchText to find points that contain text in source field.
# text is case sensitive

from qdrant_client.http import models

client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=1000,
    order_by=models.OrderBy(
        key="metadata.page",  # Field for ordering
        direction="asc"       # Change to "desc" for descending order
    ),
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.source",
                # key="metadata.page_content",
                # key="metadata.filename",
                match=models.MatchText(
                    text="AUXCA_SOP"),
            ),
        ]
    ),
)

([Record(id='a641ff3c-68ed-4cea-98fa-a927c6e5606b', payload={'metadata': {'page': 0, 'source': 'References/SOPs/AUXCA_SOP_FINAL_18JUN21_esign.pdf'}, 'page_content': 'United States Coast Guard Auxiliary  \n \n \n \n            \nAuxiliary Culinary Assistan ce (AUXCA)  \nProgram  \n \nStandard Operating Procedures'}, vector=None, shard_key=None, order_value=0),
  Record(id='b4658f4b-f88c-40d3-abf4-c6f3c0ceb315', payload={'metadata': {'page': 1, 'source': 'References/SOPs/AUXCA_SOP_FINAL_18JUN21_esign.pdf'}, 'page_content': ' \n \nComman dant \nU. S. Coast Guard  \n \n 2703 Martin Luther King Jr. Ave, SE  \nSTOP 7 501 \nWashington, DC 20593 -7501 \nStaff Symbol: CG-BSX-1 \nPhone: (202) 372 -1261 \n \n \n \n1679 0 / AUX -SOP-005(A)  \n18 Jun 21 \n \n  \nMEMORANDUM         Reply  to     S. Minutolo \n            Attn.  of:      (202) 372- 1267 \n \nFrom:  S.L. Johnson   CAPT /s/ \nCOMDT (CG -BSX)     \nTo: \n Distribution  \nSubj:  AUXILIARY CULINARY ASSISTAN CE (AUXCA)  PROGRAM STANDARD \n

##### Optional: Find points based on value-based keywords such as page number, (also works for bool)


##### Optional: Find points based on both page AND source


In [ ]:
from qdrant_client.http import models

client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=1000,
    
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.source",
                match=models.MatchText(
                    text="AUXCA_SOP"),
            ),
            models.FieldCondition(
                key="metadata.page",
                match=models.MatchValue(
                    value=6),
            ),
        ]
    ),
)

([Record(id='bfab75ea-eb39-4ebb-8bfe-6971541b5867', payload={'metadata': {'page': 6, 'source': 'References/SOPs/AUXCA_SOP_FINAL_18JUN21_esign.pdf'}, 'page_content': 'AUX -SOP-005(A) / 18 Jun 21  \n                                                                                           4                                                                  Encl (1)  \n  \n(2) To apply such support to meet needs identified by Coast Guard CS program managers and Auxiliary leadership.  \n \n(3) To improve  the morale and welf are of all Coast Guard and Auxiliary personnel \nthrough culinary service.   \n  (4) To properly train and qualify Aux iliarists  to support Coast Guard dining facilities and \ngalleys  as well as Auxiliary culinary service needs .   \n \n(5)  Trainees who pursue qualification in this  program must understand that it require s \nsignificant time commitment on their part.  Particularly, AUXCA- 1 Specialist \nassignment to a Coast Guard  cutter or shore facility could rang

## 3. Find and delete workflow

#### First set a filter that you can use in either a scroll or delete function


#### For the v1 Qdrant database, here's the best practice to be sure you get rid of all pages of a doc.

1. Run an initial search on `metadata.source` based on the file name.
2. Look at the results of the initial search to see if you are getting the doc, the whole doc, and nothing but the doc. Consider running a search on `metadata.page_content` if you are unsure.
3. Then create a final search that identifies exactly what you want to delete.
4. Confirm it's giving you the doc, the whole doc, and nothing but the doc.
5. Delete away with impunity!


In [14]:
# Defines a filter that can be used in a scroll or delete function
# This one uses MatchText to find points that contain text in source field.
# text is case sensitive

from qdrant_client.http import models

scroll_filter = models.Filter(
    must=[
        models.FieldCondition(
            # key="metadata.source",  # For metadata.source for LC objects. for non-lanchain it would just be source
            # metadata.page_content in langchain objects. For non-langchain it would just be page_content
            key="metadata.source",
            match=models.MatchText(
                text="SO"),
        ),
    ]
)

client.scroll(
    collection_name=qdrant_collection_name,
    scroll_filter=scroll_filter,
    limit=1000,
    order_by=models.OrderBy(
        key="metadata.page",  # Field for ordering
        direction="asc"       # Change to "desc" for descending order
    ),
)

([Record(id='59ed833b-d7d7-4576-9bcd-1e82bebc70a7', payload={'metadata': {'page': 0, 'source': 'References/ALAUXs/2016/01_16_AUXILIARY_MUSIC__AUXMU__SOP___31JAN2016.pdf'}, 'page_content': 'ALAUX 01 -16 - AUXILIARY MUSIC (AUXMU) SOP   \n \n31 JAN  2016  \n \nTO:      ALAUX  \nFrom:    CHDIRAUX  \nSUBJ:    AUXILIARY MUSICIAN (AUXMU) STANDARD OPERATING PROCEDURE (SOP) - \n001/16  \n1.  The Auxiliary Musician (AUXMU) Program Standard Operating Procedure (SOP) has been signed by \nthe Chief Director of Auxiliary.   This document culminates close and extensive work between Auxiliary \ngrass -roots interests that launched the initiative, Auxiliary national leadership, and the Coast Guard \nBand.   It provides programmatic structure  through which Auxiliary musicians can qualify and perform \nauthorized musical activities.  \n2.  The AUXMU SOP can be found on the CG -BSX -1 web site at: \nhttp://www.uscg.mil/auxiliary/publi cations/default.asp  , and on the Coast Guard Auxiliary web site at:  

#### Then uses the filter to DELETE the points


In [129]:
from qdrant_client.http import models

client.delete(
    collection_name=qdrant_collection_name,
    points_selector=scroll_filter,
)

UpdateResult(operation_id=189, status=<UpdateStatus.COMPLETED: 'completed'>)

### Optional: All-in-one delete function based on metadata source value


In [174]:
client.delete(
    collection_name=qdrant_collection_name,
    points_selector=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.source",
                match=models.MatchText(
                    text="011_20_WEBINAR_DELIVERY_OF_THE_TCT_REFRESHER___COVID_19")
            ),
        ]
    ),
)

UpdateResult(operation_id=209, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
content

## Example of a Qdrant Point (ie. a record)

#### A Qdrant point (ie., record) will always have an id, payload and vector field, While payload and vector are optional for qdrant, they are always populated in my case


In [ ]:
[Record(id='00034c283b0e46c9acb88308fee2001c', payload={'page_content': "*All ALAUX's are posted on the Chief Director of Auxiliary web site located at:   CHDIRAUX \nALAUX    \n \nIf you have a question regarding this ALAUX, please seek resolution within your Chain of \nLeadership and Management (COLM) including up to your servicing District Director of \nAuxiliary (DIRAUX). If your questi on still cannot be resolved after that, then please email \nCGAUX@uscg.mil .", 'metadata': {'source': '/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/data/PDF_metadata_complete/ALAUXs/012_22_AUXDATA_II_SECURITY_UPDATES_FINAL.pdf', 'page': 1, '/Producer': 'pypdf', '/Title': '012 22 AUXDATA II SECURITY UPDATES FINAL', '/LeadershipScope': '1_National', '/PageCount': '2', '/CreationDate': '2022-03-11', '/EffectiveDate': '2022-03-11', '/IngestDate': '2023-10-30', '/ExpirationDate': '2032-03-10', '/AuxSpecific': 'True', '/PublicRelease': 'True', '/PublicationNumber': 'ALAUX_012 22', '/Source': 'cgaux.org', '/Organization': 'nan', '/Curator': 'Drew_Wilkins', '/DocId': '2a91c994e8ac98ff0a2222a7b7df14ea', '/FileName': '012_22_AUXDATA_II_SECURITY_UPDATES_FINAL'}}, vector=None),
 Record(id='002794e80e7545a486d5ad536e2250bf', payload={'page_content': 'U.S COAST GUARD AUXILIARY  - UNCLASSIFIED \nThe Risk Management (RM) Instruction includes: \n•A 5-step process \n•The PEACE and STAAR models \n•Risk Assessment Matrix (RAM) \n•Mandates the use of GAR 2.0 \n•Standardizes RM training for all communities (surface, \nair, and shore) \nResponse Directorate - Telecommunications Division 7Risk Management', 'metadata': {'source': '/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/data/PDF_metadata_complete/Aux National/Telecomms_TCO_Workshop Rev 1.4_2023.pdf', 'page': 6, '/Producer': 'pypdf', '/Title': '2023 Telecomms Workshop', '/LeadershipScope': '1_National', '/PageCount': '65', '/CreationDate': '2023-10-30', '/EffectiveDate': '2023-10-30', '/IngestDate': '2023-10-30', '/ExpirationDate': 'nan', '/AuxSpecific': 'True', '/PublicRelease': 'True', '/PublicationNumber': 'nan', '/Source': 'cgaux.org', '/Organization': 'nan', '/Curator': 'Drew_Wilkins', '/DocId': '7a54199383189945b5d30bffb6ccd438', '/FileName': 'Telecomms_TCO_Workshop Rev 1'}}, vector=None)]

In [ ]:
page_number = data["metadata"]["page"]

## Example Qdrant Payload

#### If you used Langchain, the metadata field will have a page and a source. In some casee, I have enriched the metadata with additional fields like the filename.


In [ ]:
{
    "metadata": {
        "page": 162,
        "source": "References/Not catalogued/MSM Vol III Personnel CMINST 16000.8B Change 2.pdf"
    },
    "page_content": "COMDTINST M16000.8B  \nUSCG Marine Safety Manual, Vol. III:  Marine Industry Personnel  \nPART A: MARINER CREDENTIALING  \nCHAPTER 12:  LICENSING FOR ENGINEERING OFFICERS  \n \nA12-2 \n B. MMD Endorsements Accompanying Licenses.    \nEngineers holding licenses that authorize service on inspected vessels of more than 2000 \nhorsepower are entitled by 46 CFR 12.02- 11(d)(2) to an MMD endorsed for any unlicensed \nrating in t he engine department. Such license holders should be encouraged to obtain an MMD \nwhen the license is issued.  In many cases an MMD is required to legally serve aboard a vessel.  (See 46 CFR 12.02- 7). \n \nC. Creditable Service.  \n \n1. Minimum Vessel Size.    \nAll servic e must be on vessels of at least 100 gross tons.  This is consistent with the \nrequirements for ratings as a qualified member of the engine department.  See section 12.C.8 for exceptions for designated duty engineers.  \n \n2. Watchstanding Requirements.    \nTraditio nally, the Coast Guard has held watchstanding experience to be an important part \nof the professional development of third and second assistant engineers. Technological and design advances over the last several years have led to the development of ships tha t \ncan operate with unattended engine rooms.  The certificates of inspection for such vessels still require licensed engineers to be on board (the engineer being on call as needed).  Since the engineers are on call, such service shall be treated as though t he license holder \nwere in charge of a watch and will be creditable for a raise in grade"
}